### Getting the data from Douban

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
import random
import yaml

In [2]:
def extract(x):
    # Extract content using class or text
    reviews = [i.text.replace('\n', '') for i in x.find_all('span', {'class':'short'})]
    dates = [i.text.replace('\n', '').strip() for i in x.find_all('span', {'class':'comment-time'})]
    ratings = [int(str(i.find_next())[20]) for i in x.find_all(text = '看过')]
    votes = [int(i.text) for i in x.find_all('span', {'class':'votes'})]
    # Create dictionary to store scraped info
    for item in zip(reviews, dates, ratings, votes):
        scraped_info = {
        'review': item[0],
        'created_at': item[1],
        'rating': item[2],
        'votes': item[3]
        }
        yield scraped_info

In [3]:
#lifesaver: https://stackoverflow.com/questions/23102833/how-to-scrape-a-website-which-requires-login-using-python-and-beautifulsoup

with open('/home/thedatacurious/credentials.yaml') as file:
    cookies = yaml.load(file, Loader=yaml.FullLoader)['douban_cookies']

In [10]:
content2 = []

for page in range(0,500,20):
        url = "https://movie.douban.com/subject/27010768/comments?start={}&limit=20&sort=new_score&status=P&percent_type=l".format(page)
        headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:75.0) Gecko/20100101 Firefox/75.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://accounts.douban.com/passport/login?source=movie',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        }

        params = (
        ('start', '{}'),
        ('amp;limit', '20'),
        ('amp;sort', 'new_score'),
        ('amp;status', 'P'),
        ('amp;percent_type', 'm'),
        )

        response = requests.get(url, headers=headers, params=params, cookies=cookies, timeout = 10)
        
        soup = BeautifulSoup(response.content, 'html.parser')
        time.sleep(random.randint(2,5))

        content2.extend(extract(soup))
        
        if page !=0 and page % 100 == 0:
            print('{} reviews have been scraped'.format(len(content2)))
            time.sleep(random.randint(5,10))


120 reviews have been scraped
220 reviews have been scraped
320 reviews have been scraped
420 reviews have been scraped


In [7]:
medium = pd.DataFrame(content)

In [9]:
medium.to_csv('../data/raw/meh_parasite_reviews.csv', index =False, sep = '\t')

In [11]:
low = pd.DataFrame(content2)
low.to_csv('../data/raw/poor_parasite_reviews.csv', index =False, sep = '\t')

In [17]:
reviews = medium.append(low)

In [20]:
reviews.head()

,review,created_at,rating,votes
0,与《燃烧》相同，反应了贫富阶级的差异和对立，揭露了底层社会的绝望和无奈；但是与《燃烧》不同的...,2019-06-10,3,7109
1,穷人能言擅骗，个个骗术精湛，有此骗术及演技，还用住在地下室？富人幼稚可欺，全家单纯无脑，被人...,2019-08-07,3,5189
2,我为批评《燃烧》而向李沧东道歉。,2019-08-07,3,2698
3,看的有点难受，不是因为影片残忍，而是——实在太硬凹了。情节上的巧合可以理解，但一个没人找得到...,2019-08-07,3,2699
4,无法原谅开门放人进来这个facile的瞬间,2019-05-30,3,858


In [21]:
reviews.to_csv('../data/raw/combined_parasite_reviews.csv', index =False, sep = '\t')